In [ ]:
import matplotlib.pyplot as plt
import matplotlib.patches as patches
from PIL import Image
import pathlib

train_images_dir = pathlib.Path("./archive/Human Bone Fractures Multi-modal Image Dataset (HBFMID)/Bone Fractures Detection") / "train" / "images"
train_labes_dir = pathlib.Path("./archive/Human Bone Fractures Multi-modal Image Dataset (HBFMID)/Bone Fractures Detection") / "train" / "labels"
train_images = list(train_images_dir.glob("*.jpg"))

class_names = ['Comminuted', 'Greenstick', 'Healthy', 'Linear', 'Oblique Displaced', 'Oblique', 'Segmental', 'Spiral', 'Transverse Displaced', 'Transverse']

fig, axes = plt.subplots(3, 3, figsize=(10, 10))
for i, ax in enumerate(axes.flat):
    img = Image.open(train_images[i])
    ax.imshow(img)
    ax.set_title(img.size)
    # ax.axis("off")

    label_path = train_labes_dir / f"{train_images[i].stem}.txt"
    if label_path.exists():
        with open(label_path, "r") as f:
            lines = f.readlines()
        
        img_width, img_height = img.size
        for line in lines:
            values = line.strip().split()
            class_id = int(values[0])
            x_center, y_center, width, height = map(float, values[1:])

            x1 = int((x_center - width / 2) * img_width)
            y1 = int((y_center - height / 2) * img_height)

            box_width = int(width * img_width)
            box_height = int(height * img_height)

            rect = patches.Rectangle((x1, y1), box_width, box_height, linewidth=2, edgecolor="r", facecolor="none")
            ax.add_patch(rect)
            ax.text(x1-20, y1-20, class_names[class_id], color="r", fontsize=12)

In [ ]:
import torch
print(f"PyTorch: {torch.__version__}")
print(f"CUDA: {torch.version.cuda}")  # Should match your installed CUDA (12.x)
print(f"CUDA available: {torch.cuda.is_available()}")  # Must be True

In [ ]:
from ultralytics import YOLO
import torch

# Check GPU
device = 'cuda' if torch.cuda.is_available() else 'cpu'
print(f"Using device: {device}")
print(f"GPU name: {torch.cuda.get_device_name(0) if device == 'cuda' else 'CPU'}")  # Show GPU info

# Load YOLO model
try:
    model = YOLO("./yolo11m.pt").to(device)
except:
    print("Custom weights not found, falling back to pretrained YOLOv8")
    model = YOLO("yolov8m.pt").to(device)  # Fallback to standard YOLOv8 medium

# Training parameters
config = {
    "data": "./archive/Human Bone Fractures Multi-modal Image Dataset (HBFMID)/Bone Fractures Detection/data.yaml",
    "epochs": 50,  # Increased from 20 as medical imaging often benefits from more epochs
    "imgsz": 640,
    "batch": 16 if device == 'cuda' else 4,  # Smaller batch for CPU
    "device": device,
    "project": "yolo_fracture_detection",
    "name": "exp1",
    "exist_ok": True,
    "optimizer": "AdamW",  # More modern optimizer
    "lr0": 0.001,  # Lower initial LR for fine-tuning
    "pretrained": True,
    "cos_lr": True,  # Cosine learning rate scheduler
    "weight_decay": 0.0005,  # Regularization
    "fliplr": 0.5,  # Horizontal flip augmentation
    "mosaic": 1.0,  # Mosaic augmentation
    "mixup": 0.2,  # Mixup augmentation
    "close_mosaic": 10,  # Disable mosaic last epochs
    "patience": 15,  # Early stopping patience
    "save_period": 5,  # Save checkpoint every 5 epochs
    "single_cls": False,  # Set True if you have only fracture/non-fracture
}

# Train the model
try:
    results = model.train(**config)
    print("Training completed successfully!")
except Exception as e:
    print(f"Training failed: {str(e)}")

In [ ]:
import torch
print(torch.__version__)             # Should be 2.3.0+
print(torch.cuda.is_available())    # Should return True
print(torch.version.cuda)           # Should show 12.1 (but works with 12.9)

In [ ]:
import torchvision
import torch
from torchvision.ops import nms

# Dummy data
boxes = torch.tensor([[0, 0, 10, 10], [0, 0, 10, 10]], dtype=torch.float32).cuda()
scores = torch.tensor([0.9, 0.8], dtype=torch.float32).cuda()

# Prova a chiamare NMS
output = nms(boxes, scores, 0.5)
print(output)